# Домашнее задание №2
В данном задании предлагается решить [контест](https://www.kaggle.com/competitions/vk-edu-bmstu-asr-competition/overview) на платформе Kaggle.
Основная цель данного задания состоит в том, чтобы студенты ознакомились с зоопарком открытых моделей для решения задачи распознавания речи, а так же попрактиковались в подходах по постпроцессингу распознанных гипотез для улучшения качества ASR.

Разрешается пользоваться всеми предобученными моделями.

**Строго запрещено размечать данные вручную!**

## Загрузка данных и инициализация модели

In [ ]:
# https://drive.google.com/file/d/1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG/view?usp=sharing # ссылка для ручной загрузки данных

In [ ]:
!gdown 1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG && tar -xf kaggle.tar

Downloading...
From (original): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG
From (redirected): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG&confirm=t&uuid=f0de545a-6992-42d8-853d-d23e2d743fb1
To: /content/kaggle.tar
100% 138M/138M [00:04<00:00, 29.1MB/s]


In [1]:
!ls -l kaggle/wav | wc -l

643


Для базового решения задания воспользуемся предобученной моделью Whisper от OpenAI. Для этого обратимся к платформе [HuggingFace](https://huggingface.co/collections/openai/whisper-release-6501bba2cf999715fd953013)

In [2]:
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import string

In [4]:
# загрузим модель
from transformers import WhisperProcessor, WhisperForConditionalGeneration

device = "cuda:3"
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
processor.feature_extractor.return_attention_mask = True
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3                                                                                              │
│   4 device = "cuda:3"                                                                            │
│   5 model_name = "openai/whisper-large-v3"                                                       │
│ ❱ 6 processor = WhisperProcessor.from_pretrained(model_name)                                     │
│   7 processor.feature_extractor.return_attention_mask = True                                     │
│   8 model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)               │
│   9                                                                                              │
│                                                                                                  │
│ /raid/alebedev/myenvtr/lib/python3.7/site-packages/transformers/processing_utils.py:184 in       │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   181 │   │   │   │   [`~feature_extraction_utils.FeatureExtractionMixin.from_pretrained`] and   │
│   182 │   │   │   │   [`~tokenization_utils_base.PreTrainedTokenizer.from_pretrained`].          │
│   183 │   │   """                                                                                │
│ ❱ 184 │   │   args = cls._get_arguments_from_pretrained(pretrained_model_name_or_path, **kwarg   │
│   185 │   │   return cls(*args)                                                                  │
│   186 │                                                                                          │
│   187 │   @classmethod                                                                           │
│                                                                                                  │
│ /raid/alebedev/myenvtr/lib/python3.7/site-packages/transformers/processing_utils.py:228 in       │
│ _get_arguments_from_pretrained                                                                   │
│                                                                                                  │
│   225 │   │   │   else:                                                                          │
│   226 │   │   │   │   attribute_class = getattr(transformers_module, class_name)                 │
│   227 │   │   │                                                                                  │
│ ❱ 228 │   │   │   args.append(attribute_class.from_pretrained(pretrained_model_name_or_path, *   │
│   229 │   │   return args                                                                        │
│   230 │                                                                                          │
│   231 │   @property                                                                              │
│                                                                                                  │
│ /raid/alebedev/myenvtr/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1835  │
│ in from_pretrained                                                                               │
│                                                                                                  │
│   1832 │   │   │   local_files_only=local_files_only,                                            │
│   1833 │   │   │   _commit_hash=commit_hash,                                                     │
│   1834 │   │   │   _is_local=is_local,                                                           │
│ ❱ 1835 │   │   │   **kwargs,                               

## Baseline подход к инференсу модели

In [ ]:
def batch_inference(model, processor, path_to_wavs, batch_size, sampling_rate=16000):
    results = {}
    wav_files = os.listdir(path_to_wavs)

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe") # generates task specific special tokens

    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)
        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

        x, x_masks = inputs["input_features"].to(device), inputs["attention_mask"].to(device)

        with torch.no_grad():
            model.eval()
            output_ids = model.generate(x, forced_decoder_ids=forced_decoder_ids, attention_mask=x_masks)

        transcribtion = processor.batch_decode(output_ids, skip_special_tokens=True)

        results.update(zip(audio_paths, transcribtion))
    return results

In [ ]:
path_to_wavs = "kaggle/wav"
batch_size = 64

In [ ]:
results = batch_inference(model=model, processor=processor, path_to_wavs=path_to_wavs, batch_size=batch_size)

100%|██████████| 11/11.0 [02:00<00:00, 10.94s/it]


## Sample submission
Подготовим пример для загрузки на платформу

In [ ]:
def dummy_postprocessing(data):
    for filename, hypo in data.items():
        hypo = hypo.strip()
        hypo = hypo.translate(str.maketrans('', '', string.punctuation))
        hypo = hypo.lower()
        data[filename] = hypo
    return data

In [ ]:
clean_data = dummy_postprocessing(results)

In [ ]:
sample_submission = pd.read_csv("kaggle/sample_submission.csv")

In [ ]:
sample_submission.head(20)

,filename
0,audio_0.wav
1,audio_1.wav
2,audio_2.wav
3,audio_3.wav
4,audio_4.wav
5,audio_5.wav
6,audio_6.wav
7,audio_7.wav
8,audio_8.wav
9,audio_9.wav


In [ ]:
sample_submission["hypo"] = sample_submission["filename"].apply(lambda x: clean_data[x])

In [ ]:
sample_submission.insert(0, "id", sample_submission.index)

In [ ]:
sample_submission.head()

,id,filename,hypo
0,0,audio_0.wav,тревожная музыка
1,1,audio_1.wav,смех
2,2,audio_2.wav,батиша жрел ракта грямли сюда иди я сказал
3,3,audio_3.wav,высоль не понял
4,4,audio_4.wav,быстро за мной


In [ ]:
sample_submission.to_csv("baseline.csv", index=False)

## Идеи по улучшению предсказаний:
* Всегда ли LAS архитектура работает на тишине корректно?
* Как насчет нормализации текста? (19.02 - девятнадцатое февраля, 20 лет - двадцать лет)
* Может быть попробовать рескоринг? (возвращаем больше, чем топ-1 гипотезу из бима)
* Может быть взять модель побольше?
## Соревнование доступно по [ссылке](https://www.kaggle.com/t/325b553468db4d01a583e29624a7685d)

In [8]:
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import string
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [25]:
device = "cuda:3"
model_name = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)

In [26]:
def batch_inference(model, processor, path_to_wavs, batch_size, sampling_rate=16000):
    results = {}
    wav_files = os.listdir(path_to_wavs)

    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)
        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

        with torch.no_grad():
            model.eval()
            logits = model(inputs.input_values.to(device)).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcribtion = processor.batch_decode(predicted_ids)

        results.update(zip(audio_paths, transcribtion))
    return results

In [27]:
path_to_wavs = "kaggle/wav"
batch_size = 64

In [28]:
results = batch_inference(model=model, processor=processor, path_to_wavs=path_to_wavs, batch_size=batch_size)


100%|██████████| 11/11.0 [00:23<00:00,  2.14s/it]


In [29]:
def dummy_postprocessing(data):
    for filename, hypo in data.items():
        hypo = hypo.strip()
        hypo = hypo.translate(str.maketrans('', '', string.punctuation))
        hypo = hypo.lower()
        data[filename] = hypo
    return data

clean_data = dummy_postprocessing(results)

In [30]:
sample_submission = pd.read_csv("kaggle/sample_submission.csv")
sample_submission["hypo"] = sample_submission["filename"].apply(lambda x: clean_data[x])
sample_submission.insert(0, "id", sample_submission.index)

In [31]:
sample_submission["hypo"] = sample_submission["hypo"].replace('', ' ')

In [32]:
sample_submission.to_csv("baseline.csv", index=False)

In [33]:
print(sample_submission["hypo"])

0                                 перпе пюминасоналнакоп
1      о котатаашкакилад ооматиланавадобанароновнапре...
2            матише жевёт батно вребя сёрдай не оказоваи
3            слышали глуболоогрудного поречкок  не будем
4                              аа быстро за мойнатааапас
                             ...                        
637    и там же получается нет них по хитрости все оч...
638    но не сдаряясь меня завотволени как шарово не ...
639    оно идет многу со временем чем обычный бумажны...
640    дед сады и насадики для божлых людей с особенн...
641    я не готову деньма какийто нехорошая хитростир...
Name: hypo, Length: 642, dtype: object


# nvidia/canary-1b

In [11]:
import os
import pandas as pd
import string
import nemo.collections.asr as nemo_asr

ModuleNotFoundError: No module named 'editdistance'

In [3]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("nvidia/canary-1b")

transcriptions = asr_model.transcribe(["file.wav"])


OSError: Can't load tokenizer for 'nvidia/canary-1b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'nvidia/canary-1b' is the correct path to a directory containing all relevant files for a Wav2Vec2Tokenizer tokenizer.